In [123]:
import tensorflow as tf
import tf_keras
import tensorflow_model_optimization as tfmot
from tf_keras import layers,losses,optimizers,Sequential
from tf_keras.models import Model
from tf_keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D,Input
from tf_keras.applications import VGG16
from tf_keras import datasets,models
import importlib


In [124]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [125]:
from importing_dataset import load_dataset
import APoZ
importlib.reload(APoZ)
from APoZ import APoZ_Algorithm
train_examples, validation_examples, num_examples, num_classes, class_names = load_dataset('horses_or_humans', 70)

In [126]:
from reformatting import reformat_image
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
train_batches = train_examples.cache().shuffle(num_examples//4).map(reformat_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(reformat_image).batch(BATCH_SIZE).prefetch(1)
train_batches

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [109]:
# Chargez le modèle
APoZ_model = models.load_model("model_experiments/model_horses_or_humans.keras")

In [128]:
# Chargez le modèle
feature_extractor = VGG16(input_shape=(IMG_SIZE + (3,)), include_top=False)
feature_extractor.trainable = False

max_pool_layer = layers.GlobalMaxPooling2D()
prediction_layer = layers.Dense(num_classes, activation="softmax")
inputs = Input(shape=IMG_SIZE + (3,))
x = feature_extractor(inputs)
x = max_pool_layer(x)
x = Dropout(0.3)(x)
outputs = prediction_layer(x)
model = Model(inputs, outputs, name="horses_or_humans_ApoZ")

model.summary()

Model: "horses_or_humans_ApoZ"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_max_pooling2d_4 (Gl  (None, 512)               0         
 obalMaxPooling2D)                                               
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 14715714 (56.14 MB)
Trainable params: 1026 (4.01 KB)
Non-trainable params: 14714688 (56.13 MB)
__

# Compile the model

In [129]:
model.compile(
    optimizer = 'adam',
    loss = losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])

In [ ]:
EPOCHS = 3
history = model.fit(train_batches,
                    epochs = EPOCHS,
                    batch_size = BATCH_SIZE,
                    validation_data = validation_batches)

Epoch 1/3
23/23 [==============================] - 134s 6s/step - loss: 20.2389 - accuracy: 0.6328 - val_loss: 2.6890 - val_accuracy: 0.8831
Epoch 2/3
23/23 [==============================] - 130s 6s/step - loss: 3.8874 - accuracy: 0.8637 - val_loss: 0.6282 - val_accuracy: 0.9610
Epoch 3/3
15/23 [==================>...........] - ETA: 33s - loss: 2.9785 - accuracy: 0.9000

In [ ]:
from compilation import create_model_checkpoint
feature_extractor.trainable = True
# Fine-tune from this layer onwards
fine_tune_at = 2

# Freeze all the layers before the `fine_tune_at` layer
for layer in feature_extractor.layers[:-fine_tune_at]:
  layer.trainable =  False

model.compile(loss=losses.SparseCategoricalCrossentropy(),
              optimizer = optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])

EPOCHS = 5
history = model.fit(train_batches,
                    epochs = EPOCHS,
                    batch_size=BATCH_SIZE,
                    validation_data = validation_batches,
                    callbacks=[create_model_checkpoint(model_name=model.name)])

In [127]:
# Charger votre modèle
testt_model = models.load_model("model_experiments/vgg16_model_horses_or_humans.keras")

# Afficher les couches du modèle pour vérifier leur structure
print("Layers in the APoZ model:")
for layer in APoZ_model.layers:
    print(layer.name)

# Vérifier que le modèle est bien fonctionnel
if isinstance(APoZ_model, models.Model):
    print("APoZ_model is a functional model")
else:
    print("APoZ_model is not a functional model")

# Accéder à la couche "sequential_1" si elle existe
sequential_1 = APoZ_model.get_layer("sequential_1")

# Créer un sous-modèle pour obtenir les activations des couches Conv2D dans "sequential_1"
activation_model = Model(
            inputs=model.input,
            outputs=[layer.output for layer in model.layers if isinstance(layer, layers.Conv2D)]
        )




TypeError: Could not deserialize class 'Functional' because its parent module tf_keras.src.models.functional cannot be imported. Full object config: {'module': 'keras.src.models.functional', 'class_name': 'Functional', 'config': {'name': 'vgg16_model_horses_or_humans.keras', 'trainable': True, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': [None, 224, 224, 3], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer_16'}, 'registered_name': None, 'name': 'input_layer_16', 'inbound_nodes': []}, {'module': 'keras', 'class_name': 'Sequential', 'config': {'name': 'sequential', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None, 'shared_object_id': 2294770063328}, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': [None, 224, 224, 3], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer_15'}, 'registered_name': None}, {'

In [105]:
threshold = 0.9
apoz = APoZ_Algorithm(APoZ_model, train_batches,threshold)

In [106]:
pruned_model = apoz.run()

Running APoZ algorithm...
Analyzing layers for APoZ calculation...
Number of input images: 719
Normalized input data.
Activation model summary:
Model: "model_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
Total params: 0 (0.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Extracting activations from the model...
23/23 [==============================] - 0s 5ms/step


IndexError: list index out of range

In [17]:
pruned_model.summary()

NameError: name 'pruned_model' is not defined